In [1]:
%run util.py

In [2]:
# N = 2
# N = 3
N = 4
# N = 5

In [3]:
# mu_0: the initial distribution; 1 x (N**2) 
# mu: the actual stationary distribution; 1 x (N**2)
# mu_1: the estimated stationary distribution 
# P: the new transition matrix (the old transition matrix is Q)
# G_1: the estimate of the gradient
# H_1: the estimate of the Hessian
# U_1: an estimated sample path of the Gaussian random vector U; length 1000

# P is the ground truth transition matrix
mu_0, mu, mu_1, P, G_1, H_1, U_1 = ChainGen(N)  
# PP is a different transition matrix for testing purposes
mu_0P, muP, mu_1P, PP, G_1P, H_1P, U_1P = ChainGen(N)  
zdump([N, mu_0, mu, mu_1, P, G_1, H_1, U_1, mu_0P, muP, mu_1P, PP, G_1P, H_1P, U_1P], \
      './data_dir/P_PP_%d.pkz'%N)

<string>:372: RuntimeWarning: covariance is not positive-semidefinite.


In [4]:
N, mu_0, mu, mu_1, P, G_1, H_1, U_1, mu_0P, muP, mu_1P, PP, G_1P, H_1P, U_1P = zload('./data_dir/P_PP_%d.pkz'%N)

In [5]:
mu

array([ 0.02880987,  0.03564585,  0.01833014,  0.03568827,  0.00165996,
        0.11457118,  0.00306333,  0.17634308,  0.0140842 ,  0.13779579,
        0.02048629,  0.01708536,  0.0739201 ,  0.00762472,  0.14757189,
        0.16731998])

In [6]:
muP

array([ 0.32592547,  0.08359032,  0.00725468,  0.04017984,  0.07120543,
        0.03112533,  0.12786533,  0.01618187,  0.04556542,  0.08649549,
        0.07598318,  0.00592204,  0.014254  ,  0.04516681,  0.00286293,
        0.02042184])

In [7]:
# n_range = range(20, 50, 10)
# n_range = range(30, 90, 20)
# n_range = range(40, 130, 30)
n_range = [50]
# n_range = range(50, 170, 40)

In [8]:
num_test_sample = 2000
beta_list = list(np.arange(0, 0.01, 0.00001)[1:-1]) + list(np.arange(0.01, 1.01, 0.05)[:-1])

In [9]:
beta_list

[1.0000000000000001e-05,
 2.0000000000000002e-05,
 3.0000000000000004e-05,
 4.0000000000000003e-05,
 5.0000000000000002e-05,
 6.0000000000000008e-05,
 7.0000000000000007e-05,
 8.0000000000000007e-05,
 9.0000000000000006e-05,
 0.0001,
 0.00011,
 0.00012000000000000002,
 0.00013000000000000002,
 0.00014000000000000001,
 0.00015000000000000001,
 0.00016000000000000001,
 0.00017000000000000001,
 0.00018000000000000001,
 0.00019000000000000001,
 0.00020000000000000001,
 0.00021000000000000001,
 0.00022000000000000001,
 0.00023000000000000001,
 0.00024000000000000003,
 0.00025000000000000001,
 0.00026000000000000003,
 0.00027,
 0.00028000000000000003,
 0.00029,
 0.00030000000000000003,
 0.00031,
 0.00032000000000000003,
 0.00033000000000000005,
 0.00034000000000000002,
 0.00035000000000000005,
 0.00036000000000000002,
 0.00037000000000000005,
 0.00038000000000000002,
 0.00039000000000000005,
 0.00040000000000000002,
 0.00041000000000000005,
 0.00042000000000000002,
 0.00043000000000000004,
 

In [10]:
eta_wc = {}
eta_Sanov = {}
test_sample = {}
for n in n_range:
    # Get sample paths of the Markov chain with length n; 
    # these paths will be the test set
    test_sample[str(n)] = []
    for idx in range(int(num_test_sample/2)):
        test_sample[str(n)].append(chain(mu, P, n))  
    for idx in range(int(num_test_sample/2)):
        test_sample[str(n)].append(chain(muP, PP, n))  
    # Get thresholds for Hoeffding's test corresponding to sample length n    
    for beta in beta_list:
        key = str(n) + '_' + str(beta)
        eta_1 = ThresWeakConv(N, beta, n, mu_0, mu, mu_1, P, G_1, H_1, U_1).ThresCal()
        eta_2 = ThresSanov(N, beta, n, mu_0, mu, mu_1, P, G_1, H_1, U_1).ThresCal()
        eta_wc[key] = eta_1
        eta_Sanov[key] = eta_2
zdump([test_sample, eta_wc, eta_Sanov], './data_dir/testSample_threshold_%d.pkz'%N)

In [11]:
test_sample, eta_wc, eta_Sanov = zload('./data_dir/testSample_threshold_%d.pkz'%N)

In [12]:
from __future__ import division

mu = np.reshape(mu, (N, N))

TPR_wc = {}
FPR_wc = {}
TPR_Sanov = {}
FPR_Sanov = {}

for n in n_range:
    for beta in beta_list:
        TP_wc = 0
        FP_wc = 0
        TP_Sanov = 0
        FP_Sanov = 0
        key = str(n) + '_' + str(beta)
        for idx in range(num_test_sample):
            KL = KL_est(test_sample[str(n)][idx], mu)
            if idx > 1000 and KL > eta_wc[key]:
                TP_wc += 1
            if idx < 1000 and KL > eta_wc[key]:
                FP_wc += 1
            if idx > 1000 and KL > eta_Sanov[key]:
                TP_Sanov += 1
            if idx < 1000 and KL > eta_Sanov[key]:
                FP_Sanov += 1
        TPR_wc[key] = TP_wc / 1000
        FPR_wc[key] = FP_wc / 1000
        TPR_Sanov[key] = TP_Sanov / 1000
        FPR_Sanov[key] = FP_Sanov / 1000
        
zdump([TPR_wc, FPR_wc, TPR_Sanov, FPR_Sanov], './data_dir/ROCpt_%d.pkz'%N)

In [13]:
TPR_wc, FPR_wc, TPR_Sanov, FPR_Sanov = zload('./data_dir/ROCpt_%d.pkz'%N)

In [14]:
TPR_wc_list = {}
FPR_wc_list = {}
TPR_Sanov_list = {}
FPR_Sanov_list = {}
        
for n in n_range:
    TPR_wc_list[str(n)] = []
    FPR_wc_list[str(n)] = []
    TPR_Sanov_list[str(n)] = []
    FPR_Sanov_list[str(n)] = []
    TPR_wc_list[str(n)].append(0)
    FPR_wc_list[str(n)].append(0)
    TPR_Sanov_list[str(n)].append(0)
    FPR_Sanov_list[str(n)].append(0)
    for beta in beta_list:
        key = str(n) + '_' + str(beta)
        TPR_wc_list[str(n)].append(TPR_wc[key])
        FPR_wc_list[str(n)].append(FPR_wc[key])
        TPR_Sanov_list[str(n)].append(TPR_Sanov[key])
        FPR_Sanov_list[str(n)].append(FPR_Sanov[key])

    TPR_wc_list[str(n)].append(1)
    FPR_wc_list[str(n)].append(1)
    TPR_Sanov_list[str(n)].append(1)
    FPR_Sanov_list[str(n)].append(1)

In [15]:
import matplotlib.pyplot as plt
import pylab
from pylab import *

ROC_WC_list = []
ROC_Sanov_list = []
style_list = ['bo-', 'gs-', 'r+-', 'c<-', 'm>-', 'y^-']
idx = 0

pylab.xlim(-0.1, 1.15)
pylab.ylim(-0.1, 1.1)

for n in n_range:
    ROC_WC_list.append(plt.plot(FPR_wc_list[str(n)], TPR_wc_list[str(n)], style_list[idx]))
    ROC_Sanov_list.append(plt.plot(FPR_Sanov_list[str(n)], TPR_Sanov_list[str(n)], style_list[idx+1]))
    idx += 2

legend_list_1 = []
legend_list_2 = []
legend_list_1.append(ROC_WC_list)
legend_list_1.append(ROC_Sanov_list)
legend_list_1 = list(np.array(legend_list_1).ravel())

for n in n_range:
    legend_list_2.append("Use threshold $\eta_n^{wc}$; $n=%d$"%n)
for n in n_range:
    legend_list_2.append("Use threshold $\eta_n^{sv}$; $n=%d$"%n)
plt.legend(legend_list_1, legend_list_2, loc=4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid()

savefig('./data_dir/ROC_%d.eps'%N)
plt.show()